---
**Chapter 5: Introduction to Computer Theory by Daniel I. A. Cohen**

In [ ]:
!pip install SNOBOL4python==0.4.5

In [ ]:
import re
from graphviz import Digraph
from itertools import product
from pprint import pformat, pprint
from IPython.display import display, Latex, Markdown
import sys
from pprint import pprint, pformat
## Thirty one (31) flavors of patterns to choose from ...
from SNOBOL4python import ε, σ, π, λ, Λ, ζ, θ, Θ, φ, Φ, α, ω
from SNOBOL4python import ABORT, ANY, ARB, ARBNO, BAL, BREAK, BREAKX, FAIL
from SNOBOL4python import FENCE, LEN, MARB, MARBNO, NOTANY, POS, REM, RPOS
from SNOBOL4python import RTAB, SPAN, SUCCESS, TAB
# Miscellaneous
from SNOBOL4python import GLOBALS, TRACE, PATTERN, STRING
from SNOBOL4python import ALPHABET, DIGITS, UCASE, LCASE, NULL
from SNOBOL4python import nPush, nInc, nPop, Shift, Reduce, Pop
# Instantiate the global variable space
GLOBALS(globals())
FA = dict()

In [ ]:
def display_dfa(fa):
    dot = Digraph()
    dot.attr(rankdir='LR')
    dot.attr(bgcolor='black')
    dot.attr('node', style='filled', fillcolor='black', fontcolor='white', color='white')
    dot.attr('edge', color='white', fontcolor='white')
    Σ = fa[0][1:]
    Δ = fa[1:]
    for i, δ in enumerate(Δ):
        q = δ[0]
        is_start = False
        is_finish = False
        if m := re.match(r"^\-(.*)$", q): is_start = True;  q = m.groups(1)[0]
        if m := re.match(r"^\+(.*)$", q): is_finish = True; q = m.groups(1)[0]
        if is_finish: dot.node(str(i+1), shape='doublecircle')
        else:         dot.node(str(i+1), shape='circle')
        if is_start:
            dot.node('', width='0', height='0', margin='0', shape='point', style='invis')
            dot.edge('', str(i+1), )
    for i, δ in enumerate(Δ):
        for j in range(1, len(δ)):
            dot.edge(str(i+1), str(δ[j]), label=Σ[j-1])
    dot.render('dfa', format='png', cleanup=False)  # Saves as dfa.png
    display(dot)

In [ ]:
def compile_fa(fa_table):
    header = fa_table[0][1:]
    states = fa_table[1:]
    start_state = None
    accepting_states = set()
    transitions = {}
    for row in states:
        state_label = row[0]
        state_num = int(state_label.lstrip('+-'))
        if state_label.startswith('-'):
            start_state = state_num
        if state_label.startswith('+') or state_label.startswith('-+'):
            accepting_states.add(state_num)
        transitions[state_num] = {}
        for symbol, next_state in zip(header, row[1:]):
            transitions[state_num][symbol] = next_state
    if start_state is None:
        raise ValueError("No start state defined in FA table.")
    return start_state, header, transitions, accepting_states

In [ ]:
def run_fsm(input_str, start_state, header, transitions, accepting_states):
    current_state = start_state
    for ch in input_str:
        if ch not in header:
            raise ValueError(f"Invalid input symbol: {ch}")
        current_state = transitions[current_state][ch]
    accepted = current_state in accepting_states
    return accepted, current_state

In [ ]:
def show_samples(table):
    accepted = []
    rejected = []
    alphabet = table[0][1:]
    fsm = compile_fa(table)
    for length in range(0, 8):
        for combo in product(alphabet, repeat=length):
            word = ''.join(combo)
            is_accepted, _ = run_fsm(word, *fsm)
            if is_accepted:
                accepted.append(word)
            else: rejected.append(word)
    return accepted, rejected

In [ ]:
import textwrap
def display_problem(problem_name, verbose=True):
    problem = FA[problem_name]
    display(Latex(problem[0]))
    display_dfa(problem[2])
    accepted, rejected = show_samples(problem[2])
    if (verbose):
        print("Accepted:")
        print(textwrap.fill(" ".join(accepted), width=80))
        print()
        print("Rejected:")
        print(textwrap.fill(" ".join(rejected), width=80))

---
**Problem 1:** Write out the transition tables for the FAs on pp. 56, 58 (both), 63, 64, and 69 that were defined by pictures.

**Answer 1:** *See below*

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_56'] = (\
  "$(a+b)^{*}b(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3]
  , [  '2',  1 ,  3]
  , [ '+3',  3 ,  3]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_56')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_1'] = (\
  "$(a+b)(a+b)^{*}$"
, SPAN('ab')
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [ '+2',  2 ,  2 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_1')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_58_2'] = (\
  "$(a+b)^{*}$"
, SPAN('ab') | ε()
, [ [  ' ', 'a', 'b']
  , ['-+1',  1 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_58_2')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_63'] = (\
  "$(a+b)^{*}(aa+bb)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b')) + (σ('aa') | σ('bb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  4 ,  3 ]
  , [  '3',  2 ,  4 ]
  , [ '+4',  4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_63')

In [ ]:
#-------------------------------------------------------------------------------
FA['pg_64'] = (\
  "$(a+b)(a+b)b(a+b)*$"
, (σ('aaa') | σ('bbb')) + ARBNO(σ('a') | σ('b'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  5 ]
  , [  '4',  4 ,  4 ]
  , [ '+5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('pg_64')

In [ ]:
# pg_69
na = nb = 0
def init(): na = nb = 0; return True
def acnt(): global na; na += 1; return True
def bcnt(): global nb; nb += 1; return True
FA['EVEN_EVEN'] = (\
  "$(aa+bb+(ab+ba)(aa+bb)^{*}(ab+ba))^{*}$"
, POS(0) + Λ("init()")
+ ARBNO(
    σ('a') + λ("acnt()")
  | σ('b') + λ("bcnt()")
  )
+ RPOS(0) + λ("a % 2 == 0 and b % 2 == 0")
, [ [  ' ', 'a', 'b']
  , ['-+1',  3 ,  2 ]
  , [  '2',  4 ,  1 ]
  , [  '3',  1 ,  4 ]
  , [  '4',  2 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('EVEN_EVEN')

---
**Problem 2:** Build an FA that accepts only the language of all words with b as the second letter. Show both the picture and the transition table for this machine and find a regular expression for the language.

**Answer 2:**

RE = $(a + b)b(a+b)^{*}$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_2'] = (\
  "$(a+b)b(a+b)^{*}$"
, ANY("ab") + σ('b') + ARBNO(σ('a') | σ('b'))
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  2 ]
  , [  '2', 3 ,  4 ]
  , [  '3', 3 ,  3 ]
  , [ '+4', 4 ,  4 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_2')

---
**Problem 3:** Build an FA that accepts only the words baa, ab, and abb and no other strings longer or shorter.

**Answer 3:**

RE = $(ab(Λ + b)+baa)$

In [ ]:
#-------------------------------------------------------------------------------
FA['prob5_3'] = (\
  "$(ab(Λ + b)+baa)$"
, σ('baa') | σ('ab') | σ('abb')
, [ [  ' ','a', 'b']
  , [ '-1', 2 ,  5 ]
  , [  '2', 8 ,  3 ]
  , [ '+3', 8 ,  4 ]
  , [ '+4', 8 ,  8 ]
  , [  '5', 6 ,  8 ]
  , [  '6', 7 ,  8 ]
  , [ '+7', 8 ,  8 ]
  , [  '8', 8 ,  8 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_3')

---
**Problem 4:**
* (i) Build an FA with the three states that accepts all strings.
* (ii) Show that given any FA with three states and three *+*'s, it accepts all input strings.
* (iii) If an FA has three states and only one *+*, must it reject some inputs?

**Answer 4:**
- *See below*
- follows from the pigeon-hole principle
- No.

In [ ]:
FA['prob5_4'] = (\
  "$(a+b)(a+b)(a+b)^{*}$"
, ARBNO(σ('a') | σ('b'))
, [ [ ' ' , 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  3 ,  3 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_4')

---
**Problem 5:**
* (i) Build an FA that accepts only those words that have more that four letters.
* (ii) Build an FA that accepts only those words that have fewer than four letters.
* (iii) Build an FA that accepts only those words with exactly four letters.

**Answer 5:**
- RE: $\quad (a+b)^{5}(a+b)^{*}$
- RE: $\quad Λ+a+b+(Λ+a+b+(Λ+a+b)), \quad (Λ+a+b)^3$
- RE: $\quad (a+b)(a+b)(a+b)(a+b), \quad (a+b)^4$

In [ ]:
FA['prob5_5_1'] = (\
  "$(a+b)(a+b)(a+b)(a+b)(a+b)(a+b)^{*}$"
, SPAN("ab") @ "tx" + Λ("len(tx) > 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [  '5',  6 ,  6 ]
  , [ '+6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_1')

In [ ]:
FA['prob5_5_2'] = (\
  "$(Λ+a+b)(Λ+a+b)(Λ+a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) < 4")
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  2 ]
  , [ '+2',  3 ,  3 ]
  , [ '+3',  4 ,  4 ]
  , [ '+4',  5 ,  5 ]
  , [  '5',  5 ,  5 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_2')

In [ ]:
FA['prob5_5_3'] = (\
  "$(a+b)(a+b)(a+b)(a+b)$"
, SPAN("ab") @ "tx" + Λ("len(tx) == 4")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  3 ]
  , [  '3',  4 ,  4 ]
  , [  '4',  5 ,  5 ]
  , [ '+5',  6 ,  6 ]
  , [  '6',  6 ,  6 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_5_3')

---
**Problem 6:** Build an FA that accepts only those words that do *not* end with *ba*.

**Answer 6:**
RE: $Λ+a+(a+b)^{*}(b+aa)$

In [ ]:
FA['prob5_6'] = (\
  "$Λ+a+(a+b)^{*}(b+aa)$"
, ARBNO(σ('a') + σ('b')) + σ('ba') + RPOS(0) + FAIL()
, [ [  ' ' , 'a', 'b']
  , ['-+1' ,  1 ,  2 ]
  , [ '+2' ,  3 ,  2 ]
  , [  '3' ,  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_6')

---
**Problem 7:** Build an FA that accepts only those words that begin or end with a double letter.

**Answer 7:** RE: $\quad (aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$

In [ ]:
FA['prob5_7'] = (\
  "$(aa+bb)(a+b)^{*}+(a+b)^{*}(aa+bb)$"
, ( POS(0) + (σ('aa') | σ('bb')) + ARB()
  | (σ('aa') | σ('bb')) + ARB() + RPOS(0)
  )
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  4 ]
  , [   '2',  3 ,  6 ]
  , [  '+3',  3 ,  3 ]
  , [   '4',  7 ,  5 ]
  , [  '+5',  5 ,  5 ]
  , [   '6',  7 ,  8 ]
  , [   '7',  9 ,  6 ]
  , [  '+8',  7 ,  8 ]
  , [  '+9',  9 ,  7 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_7')

---
**Problem 8:** Build an FA that accepts only those words that have an even number of substrings *ab*.

**Answer 8: ???**
- RE: $\quad (abab)^{*}$
- RE: $\quad (aa^{*}bb^{*}aa^{*}bb^{*})^{*}$

In [ ]:
FA['prob5_8_1'] = (\
  "$(abab)^{*}$"
, ARBNO(σ('abab'))
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  6 ]
  , [  '2',  6 ,  3 ]
  , [  '3',  4 ,  6 ]
  , [  '4',  6 ,  5 ]
  , [ '+5',  2 ,  6 ]
  , [  '6',  6 ,  6 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_8_1')

In [ ]:
FA['prob5_8_2'] = (\
  "$(aa^{*}bb^{*}aa^{*}bb^{*})^{*}$"
, ARBNO(
    σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  + σ('a') + ARBNO(σ('a')) + σ('b') + ARBNO(σ('b'))
  )
, [ [   ' ', 'a', 'b']
  , [ '-+1',  2 ,  1 ]
  , [   '2',  2 ,  3 ]
  , [   '3',  4 ,  3 ]
  , [   '4',  4 ,  1 ]
  ])
#-------------------------------------------------------------------------------
display_problem('prob5_8_2')

---
**Problem 9:**
* (i) Recall from Chapter 4 the language of all words over the alphabet {*a b*} that have both the letter *a* and the letter *b* in them, but not necessarily in that order. Build an FA that accepts this language.
* (ii) Build an FA that accepts the language of all words with only *a*'s or only *b*'s in them. Give a regular expression for this language.

**Answer 9:**
- (i) RE: $(a+b)^{*}(ab+ba)(a+b)^{*}$ or $\quad (aa^{*}b+bb^{*}a)(a+b)^{*}$
- (ii) RE: $aa^{*}+bb^{*}$


In [ ]:
FA['prob5_9_1'] = (\
  "$(a+b)^{*}(ab+ba)(a+b)^{*}$"
, ARB() + σ('a') + ARB() + σ('b') + ARB()
| ARB() + σ('b') + ARB() + σ('a') + ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  4 ]
  , [  '3',  4 ,  3 ]
  , [ '+4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_1')

In [ ]:
FA['prob5_9_2'] = (\
  "$aa^{*}+bb^{*}$"
, SPAN("a") | SPAN("b")
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [ '+2',  2 ,  4 ]
  , [ '+3',  4 ,  3 ]
  , [  '4',  4 ,  4 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_9_2')

---
**Problem 10:** Consider all the possible FAs over the alphabet { a, b } that have exactly two states. An FA must have a designated start state, but there are four possible ways to place the *+*'s:

Type 1: ['-',  '']
Type 2: ['-+', '']
Type 3: ['-',  '+']
Type 4: ['-+', '+']

Each FA needs four edges (two from each state), each of which can lead to either of the states. There are $2^{4}=16$ ways to arrange the labeled edges for each of the four types of FAs. Therefore, in total there are 64 different FAs of two states. However, they do not represent 64 non-equivalent FAs because they are not all associated with different languages. All type 1 FAs do not accept any words at all, whereas all FAs of type 4 accepts all strings of *a*'s and *b*'s.

* (i) Draw the remaining FAs of type 2.
* (ii) Draw the remaining FAs of type 3.
* (iii) Recalculate the total number of two-state machines using the transition table definition.

**Book answer 10:**

* (i-ii) *See below*
* (iii) Transition tables for a two-state machine has four cells, two rows by two columns. Each may be filled with 1 or 2. So there are $2^4$ possibilities. There are still the same four ways to designate final states. Hence the total number of different transition tables is, surprise, 64, 2^4*4.

In [ ]:
import itertools
FA_template = (\
  ""
, ARB()
, [ [ ' ', 'a', 'b']
  , [ '1',  0 ,  0 ]
  , [ '2',  0 ,  0 ]])
#-------------------------------------------------------------------------------
for t in range(1,3):
    for i, combo in enumerate(itertools.product([1, 2], repeat=4)):
        name = f"prob_10_{i}"
        if t == 1:
            FA_template[2][1][0] = '-+1'
            FA_template[2][2][0] = '2'
        if t == 2:
            FA_template[2][1][0] = '-1'
            FA_template[2][2][0] = '+2'
        FA_template[2][1][1] = combo[0]
        FA_template[2][1][2] = combo[1]
        FA_template[2][2][1] = combo[2]
        FA_template[2][2][2] = combo[3]
        FA[name] = FA_template
        display_problem(name)

---
**Problem 11:** Show that there are exactly 5832 different finite automata with three states *x* *y*, *z* over the alphabet {*a b*}, where *x* is always the start state.

**Answer 11:** $1*2^3*3^6=5832$

---
**Problem 12:** Suppose a particular FA, called FIN, has the property that it had only one final state that was not the start state. During the night, vandals come and switch the *+* sign with the *-* sign and reverse the direction of all the edges.
* (i) Show that the picture that results might not actually be an FA at all by giving an example.
* (ii) Suppose, however, that in a particular case what resulted was, in fact, a perfectly good FA. Let us call it NIF. Give an example of one such machine.
* (iii) What is the relationship between the language accepted by FIN and the language accepted by NIF as described in part (ii)? Why?
* (iv) One of the vandals told me that if in FIN the plus state and the minus state were the same state, then the language accepted by the machine could contain only palindromic words. Defeat this vandal by example.

**Answer 12:**
* (i) Any FA with a state having more than one incoming of the same character is problematic
* (ii) Ensuring one of each in and out for each character of every state
* (iii) The language accepted by NIF is the reverse of every word in the language accepted by FIN. For every path from one state to another in FIN, the exact reverse path exists in NIF.

In [ ]:
FA['prob5_12_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  2 ,  3 ]
  , [ '+3',  3 ,  3 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_1')

In [ ]:
FA['prob5_12_2'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [ '+3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_2')

In [ ]:
FA['prob5_12_3'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , ['-+1',  2 ,  3 ]
  , [  '2',  3 ,  1 ]
  , [  '3',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_12_3')

---
**Problem 13:** We define a removable state as a state such that if we erase the state itself and the edges that come out of it, what results is a perfectly good-looking FA.
* (i) Give an example of an FA that contains a removable state.
* (ii) Show that if we erase a removable state the language defined by the reduced FA is exactly the same as the language defined by the old FA.

**Answer 13:**

* (i) *See below*
* (ii) An FA only retains its properties if the state being removed could not actually be reached, that is a state is removable only if there are no edges coming into it. Clearly if the stae is never used it has no bearing on the language that the FA accepts.

In [ ]:
FA['prob5_13_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  1 ,  1 ]
  , [ '+2',  1 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_13_1')

---
**Problem 14:**
* (i) Build an FA that accepts the language of all strings of *a*'s and *b*'s such that the next to the last letter is an *a*.
* (ii) Build an FA that accepts the language of all strings of length 4 or more such that the next-to-last letter is equal to the second letter of the input string.

**Answer 14:** *See below*

In [ ]:
FA['prob5_14_1'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  1 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  3 ,  4 ]
  , [ '+4',  3 ,  1 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_14_1', verbose=True)

In [ ]:
FA['prob5_14_2'] = (\
  ""
, ARB()
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  7 ]
  , [  '3',  4 ,  3 ]
  , [  '4',  5 ,  6 ]
  , [ '+5',  5 ,  6 ]
  , [ '+6',  5 ,  3 ]
  , [  '7',  7 ,  8 ]
  , [  '8', 10 ,  9 ]
  , [ '+9', 10 ,  9 ]
  , ['+10',  7 ,  9 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_14_2', verbose=True)

---
**Problem 15:** Build a machine that accepts all strings that have an even length that is not divisible by 6.

---
**Problem 16:** Build an FA such that when the labels *a* and *b* are swapped the new machine is different from the old one but equivalent (the language defined by these machines is the same).

**Answer 16:** *See below*

In [ ]:
FA['prob5_16'] = (\
  "$(aa+ab+ba+bb)a$"
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  _ ,  _ ]
  , [ '+2',  _ ,  _ ]
  , [  '3',  _ ,  _ ]
  , [  '4',  _ ,  _ ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_1')

---
**Problem 17:**
* (i-iii) Describe in English the languages accepted by the following FAs.
* (iv) Write regular expressions for the languages accepted by these three machines.

**Answer 17:**
* (i) odd-length sequence ending in *a*
* (ii) non-null even-length sequence ending in *a*
* (iii) non-null even-length sequence of *xa*
* (iv) *See below*


In [ ]:
FA['prob5_17_1'] = (\
  "$(aa+ab+ba+bb)a$"
, ARB
, [ [  ' ', 'a', 'b']
  , [  '1',  2 ,  2 ]
  , [ '-2',  3 ,  1 ]
  , [ '+3',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_1')

In [ ]:
FA['prob5_17_2'] = (\
  "$(a+b)(aa+ab+ba+bb)a$"
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  4 ]
  , [ '+3',  2 ,  2 ]
  , [  '4',  2 ,  2 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_2')

In [ ]:
FA['prob5_17_3'] = (\
  "$(aa+ba)(aa+ba)^{*}$"
, ARB
, [ [  ' ', 'a', 'b']
  , [ '-1',  2 ,  2 ]
  , [  '2',  3 ,  5 ]
  , [ '+3',  4 ,  4 ]
  , [  '4',  3 ,  5 ]
  , [  '5',  5 ,  5 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_17_3')

---
**Problem 18:** The following is an FA over the alphabet $\Sigma = \{ a\, b\, c \}$. Prove that it acepts all strings that have an odd number of ocurrences of the substring *abc*.

**Answer 18:** The machine will stay in states 1, 2, or 3 until *abc* is seen leading to state 4. The machine will stay in states 4, 5, or 6 until another *abc* is seen leading to state 1, and repeat. So no occurence of *abc* seen while in states 1-3, and one occurrence seen while in states 4-6. Due to repitition, this translates to even and odd. All states 4-6 are final states.

In [ ]:
FA['prob5_18'] = (\
  ""
, ARB
, [ [  ' ', 'a', 'b', 'c']
  , [ '-1',  2 ,  1 ,  1 ]
  , [  '2',  2 ,  3 ,  1 ]
  , [  '3',  2 ,  1 ,  4 ]
  , [ '+4',  5 ,  4 ,  4 ]
  , [ '+5',  5 ,  6 ,  4 ]
  , [ '+6',  5 ,  4 ,  1 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_18')

---
**Problem 19:** Consider the following FA:
* (i) Show that any input string with more than three letters is not accepted by this FA.
* (ii) Show that the only words accepted are *a*, *aab*, and *bab*.
* (iii) Show that by changing the location of *+* signs, we can make this FA accept the language { *bb* *aba* *bba* }.
* (iv) Show that any language in which the words have fewer than four letters can be accepted by a machine that looks like this one with the *+* signs in different places.
* (v) Prove that if L is a finite language, then there is some FA that accepts L extending the binary-tree part of this machine several more layers if necessary.

**Book answer 19:**
* (i) This FA starts out like a binary tree. For the first three letters of the input, each time we read a letter we reach some node on the next level. However reading any more letters traps us in the dead end state at the bottom. So no word of more than three letters can be accepted.
* (ii) There are exactly three final states in the machine. Concatenation of the labels on the transitions to these states shows that the machine accepts the words *a*, *aab*, and *bab*.
* (iii) Eliminate the three existing pluses. Place new pluses in each of the following states, the right-most node on level two, the third node from the left on level three, and the second node from the right also on level three.
* (iv) Each of the 15 possible strings with less than four letters is represented by one of the nodes (excluding the dead end state). Note the lexicographic order. Simply mark those nodes that represent the words in the language with plus signs.
* (v) Since L is finite, we can determine the longest word and then build a binary tree large enough to accommodate it. Next we add a dead end state with transitions from all the leaves and itself. For every word in L, we place a plus in the appropriate, unique state.

In [ ]:
FA['prob5_19'] = (\
  ""
, ARB
, [ [   ' ', 'a', 'b']
  , [  '-1',  2 ,  3 ]
  , [   '2',  4 ,  5 ]
  , [   '3',  6 ,  7 ]
  , [   '4',  8 ,  9 ]
  , [   '5', 10 , 11 ]
  , [   '6', 12 , 13 ]
  , [   '7', 14 , 15 ]
  , [   '8', 16 , 16 ]
  , [   '9', 16 , 16 ]
  , [  '10', 16 , 16 ]
  , [  '11', 16 , 16 ]
  , [  '12', 16 , 16 ]
  , [  '13', 16 , 16 ]
  , [  '14', 16 , 16 ]
  , [  '15', 16 , 16 ]
  , [  '16', 16 , 16 ]])
#-------------------------------------------------------------------------------
display_problem('prob5_19')

---
**Problem 20:** Let us consider the possibility of an infinite automaton that starts with this infinite binary tree. Let L be any infinite language of strings of *a*'s and *b*s whatsoever. Show that by the judicious placement of *+*'s, we can turn the picture above into an infinite automaton to accept the language L. Show that for any given finite string, we can determine from this machine, in a finite time, whether it is a word in L. Discuss why this machine would not be a satisfactory language-definer for L.

**Book answer 20:** Even though the binary tree is infinite, the machine retains the uniqueness property (of the machines described in problem 19) that the path for every string ends in a different state. Surely just by placing the plus signs in the nodes that represent words in L would constitute an infinite automata. However, the machine obviously fails to define the language because no one can write down the infintely large picture.